In [3]:
import os
import json
from elasticsearch import Elasticsearch
from google.cloud import secretmanager
from dotenv import load_dotenv

# --- Prerequisite: Helper function to get secrets ---
# This is copied from your agent.py to make the script runnable.
# Ensure you have a .env file with GOOGLE_CLOUD_PROJECT defined
# or have the environment variable set.
load_dotenv()

def get_secret(secret_id: str, version_id: str = "latest") -> str:
    """
    Retrieves a secret's value from Google Cloud Secret Manager.
    """
    project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
    if not project_id:
        raise ValueError("GOOGLE_CLOUD_PROJECT environment variable not set.")
        
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    
    try:
        response = client.access_secret_version(request={"name": name})
        # Decode the payload AND strip any leading/trailing whitespace or newlines
        return response.payload.data.decode("UTF-8").strip()
    except Exception as e:
        print(f"Error accessing secret '{secret_id}': {e}")
        raise

# --- Main Test Function ---
def test_elasticsearch_search():
    """
    Connects to Elasticsearch and performs a simple semantic search.
    """
    print("--- Starting Elasticsearch Test ---")
    
    try:
        # 1. Initialize the Elasticsearch client
        print("Connecting to Elasticsearch...")
        api_key = get_secret("ELASTIC_API_KEY")
        client = Elasticsearch(
            "https://ask-g2m-c7a0bf.es.us-east-1.aws.elastic.cloud:443",
            api_key=api_key,
        )
        print("Connection successful.")

        # 2. Define a test query
        test_query = "cloud services"
        print(f"Performing semantic search for: '{test_query}'")

        # 3. Construct the retriever object and perform the search
        retriever_object = {
            "standard": {
                "query": {
                    "semantic": {
                        "field": "text", 
                        "query": test_query
                    }
                }
            }
        }
        search_response = client.search(
            index="ask-g2m-serverless", 
            retriever=retriever_object, 
            size=3  # Get the top 3 results
        )

        # 4. Print the results
        hits = search_response.get('hits', {}).get('hits', [])
        
        print(f"\n--- Search Results ---")
        print(f"Found {len(hits)} hits.")
        
        if not hits:
            print("No documents were found. The connection works, but the query returned no results.")
        else:
            for i, hit in enumerate(hits):
                print(f"\n--- Hit {i+1} ---")
                print(f"Score: {hit.get('_score')}")
                # Pretty print the source document
                print("Source Document:")
                print(json.dumps(hit.get('_source', {}), indent=2))

    except Exception as e:
        print(f"\n--- AN ERROR OCCURRED ---")
        print(f"Test failed: {e}")

# --- To run the test ---
if __name__ == "__main__":
    test_elasticsearch_search()

--- Starting Elasticsearch Test ---
Connecting to Elasticsearch...
Connection successful.
Performing semantic search for: 'cloud services'

--- Search Results ---
Found 3 hits.

--- Hit 1 ---
Score: 22.48539
Source Document:
{
  "text": "Cloud Provider Service",
  "page_number": "Unknown",
  "file_name": "dir.texas.gov_contract_DIR-CPO-STS-585_0000_00_00(1).docx"
}

--- Hit 2 ---
Score: 20.132591
Source Document:
{
  "text": "DCS\u00a0Public Cloud Services",
  "page_number": "Unknown",
  "file_name": "dir.texas.gov_contract_data_center_services_0000_00_00(1).docx"
}

--- Hit 3 ---
Score: 19.777613
Source Document:
{
  "text": "TEXAS DEPARTMENT OF INFORMATION RESOURCES | DECEMBER 2014 3 \nCloud Services Models \nThere are three basic cloud service models that are useful  for different agency needs. \n \n\uf0a1 Software as a Service (SaaS) delivers applications, such as email, customer relationship \nmanagement, and collaboration software. \n\uf0a1 Platform as a Service (PaaS) delivers a